In [ ]:
import clip #for model classification
import torch
from PIL import Image
import requests
from pymongo import MongoClient
from io import BytesIO
from tqdm import tqdm#lib to show progress



In [3]:
from dotenv import load_dotenv
import os

In [4]:
load_dotenv()
db_url=os.getenv("DB_URL")

In [5]:
db_name="Honda_cars"
db_collection="listing"

In [ ]:
def get_image_labels(url):
    